### SMS Spam classification

In [210]:
import pandas as pd
url = 'https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/10.%20RNN%20Text%20Classification/dataset/spam.csv'


In [211]:
df = pd.read_csv(url, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


- Preprocessing

In [212]:
# Selection
df = df[['v1', 'v2']]

In [213]:
# 결측치 확인
df.isna().sum().sum()

0

In [214]:
# 중복 데이터 확인
print(df.shape, df.v2.nunique())

(5572, 2) 5169


In [215]:
# 중복 데이터 제거
df.drop_duplicates(subset=['v2'], inplace=True)
df.shape


(5169, 2)

In [216]:
# Label Encoding, ['ham', 'spam'] --> [0, 1]
df.v1 = df.v1.replace(['ham', 'spam'], [0, 1])
df.head(3)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [217]:
# Ham/Spam 분포
df.v1.value_counts()

v1
0    4516
1     653
Name: count, dtype: int64

- 텍스트 전처리

In [218]:
# 구둣점, 숫자 제거
df.v2 = df.v2.str.replace('[^A-Za-z]', ' ', regex=True)
df.v2[0]

'Go until jurong point  crazy   Available only in bugis n great world la e buffet    Cine there got amore wat   '

In [219]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.v2.values, df.v1.values, stratify=df.v1.values, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4135,), (1034,), (4135,), (1034,))

- Text Encoding

In [220]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')


In [221]:
# ❌Never do that!!❌
# different vocabulary in train and test
X_train_cv = cvect.fit_transform(X_train)
X_test_cv = cvect.fit_transform(X_test)

In [222]:
# 변환된 단어 세트가 train/test dataset에서 동일해야 함
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((4135, 6494), (1034, 6494))

- 학습 및 평가

In [223]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression()

In [224]:
lrc.fit(X_train_cv, y_train)

LogisticRegression()

In [225]:
lrc.score(X_test_cv, y_test)

0.9709864603481625

- Bigram

In [226]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape


((4135, 28822), (1034, 28822))

In [227]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_cv2, y_train)
lrc.score(X_test_cv2, y_test)

0.9680851063829787

- TfidfVectorizer

In [228]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((4135, 6494), (1034, 6494))

In [229]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_tv, y_train)
lrc.score(X_test_tv, y_test)

0.9458413926499033

- TfidfVectorizer + Bigram

In [230]:
tvect2 = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((4135, 28822), (1034, 28822))

In [231]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_tv2, y_train)
lrc.score(X_test_tv2, y_test)

0.9410058027079303

- 실제 데이터로 검증

In [232]:
df = pd.read_csv(url, encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [233]:
sms = [df.v2[0], df.v2[2]]
sms

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]

In [234]:
lrc = LogisticRegression(random_state=2023)
lrc.fit(X_train_cv, y_train)

LogisticRegression(random_state=2023)

In [235]:
# 텍스트 전처리, 숫자와 구둣점 제거
import re
sms = map(lambda x: re.sub('[^A-Za-z]', ' ', x), sms)



In [236]:
# feature 변환
sms_cv = cvect.transform(sms)
sms_cv.shape

(2, 6494)

In [237]:
# 예측
lrc.predict(sms_cv)

array([0, 1], dtype=int64)

💯데이터 전처리 - 텍스트 전처리 - 데이터셋 분할 - feature 변환 - 분류기 학습/평가

- 베스트 파라메터 찾기

In [238]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [239]:
pipeline = Pipeline([('CVECT', cvect), ('LRC', lrc)])
lrc = LogisticRegression(random_state=2023)
params = {
    'CVECT__ngram_range': [(1,1), (1,2)],
    'LRC__C': [0.1, 1, 10]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 14.3 s
Wall time: 4.6 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT',
                                        CountVectorizer(stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LRC__C': [0.1, 1, 10]},
             scoring='accuracy')

In [240]:
grid_pipe.best_params_

{'CVECT__ngram_range': (1, 1), 'LRC__C': 10}

In [241]:
lrc = LogisticRegression(random_state=2023)
params = {
    'CVECT__ngram_range': [(1,1), (1,2)],
    'LRC__C': [5, 8, 10, 12, 20]
}
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3)
%time grid_pipe.fit(X_train, y_train)

CPU times: total: 26 s
Wall time: 7.54 s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT',
                                        CountVectorizer(stop_words='english')),
                                       ('LRC',
                                        LogisticRegression(random_state=2023))]),
             param_grid={'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LRC__C': [5, 8, 10, 12, 20]},
             scoring='accuracy')

In [242]:
grid_pipe.best_params_

{'CVECT__ngram_range': (1, 1), 'LRC__C': 8}

In [243]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.9738878143133463

### IMDB 영화평 감성분석(이진 분류)
- Kaggle - Bag of Words Meets Bags of Popcorn

##### 1. 데이터 탐색

In [244]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/labeledTrainData.tsv', sep='\t', quoting=3)  # 3: Quote None
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [246]:
print(df.review[0][:1000])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [247]:
# 결측치 체크
df.isna().sum().sum()

0

In [248]:
# 중복 여부 체크
df.review.nunique()
 

24904

In [249]:
# 중복 데이터 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(24904, 3)

##### 2. Text preprocessing

In [250]:
# <br /> 태그는 공백으로
df.review = df.review.str.replace('<br />', ' ')

In [251]:
# 구두점, 숫자 제거 - 영문자 이외의 문자 공백처리
df.review = df.review.str.replace('[^A-Za-z]', ' ', regex=True)

##### 3. split datasets

In [252]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review.values, df.sentiment.values, stratify=df.sentiment.values, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19923,), (4981,), (19923,), (4981,))

##### 4. Text encoding

In [253]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [254]:
# train과 test dataset의 변환후 사이즈가 동일해야 함
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((19923, 66641), (4981, 66641))

##### 5. 학습 및 평가

In [255]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2023, max_iter=500)

In [256]:
%time lrc.fit(X_train_cv, y_train)

CPU times: total: 22 s
Wall time: 6.02 s


LogisticRegression(max_iter=500, random_state=2023)

In [257]:
lrc.score(X_test_cv, y_test)

0.8813491266813893

##### 6. Bigram

In [258]:
cvect2 = CountVectorizer(stop_words='english', ngram_range=(1,2))

cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((19923, 1454639), (4981, 1454639))

In [259]:
lrc2 = LogisticRegression(random_state=2023, max_iter=500)

%time lrc2.fit(X_train_cv2, y_train)
lrc2.score(X_test_cv2, y_test)

CPU times: total: 3min 42s
Wall time: 59.3 s


0.8958040554105601

##### 7. 변환기/모델 저장/로드

In [260]:
import joblib

In [261]:
# 변환기/모델 저장
joblib.dump(cvect2, 'model/imdb_cvect2.pkl')
joblib.dump(lrc2, 'model/imdb_lrc2.pkl')

['model/imdb_lrc2.pkl']

In [262]:
# 변환기/모델 로드
new_cvect = joblib.load('model/imdb_cvect2.pkl')
new_lrc = joblib.load('model/imdb_lrc2.pkl')

##### 8. 실제 데이터로 검증

In [263]:
reviews = [
'''
This is the type of storyline that remains jn our hearts for a lifetime.
Movies like these are my favorite. SciFi, AI, post-apocalyptic (ish), with amazing cinematography,
fantastic actors, and great writing. The Creator starts off with a great hook and it doesn't let go of you.
I was immediately immersed in this world of AI and humans living as independent creatures fighting for survival,
both entities in their own way trying to survive, coexist, and overcome. I truly believe this will be a tale of the ages.
If you like movies like "AI Robot" with Will Smith, "Chappie", or even Disney's Wall-E,
then this is another epic tale to add to your list.''',
'''OMG, the old holes are horrific. This is one of the most disjointed movies of all time.
It's like 50 short films stiched together. The story jumps all over the place and you have to try and join the dots.
Either that or there is an entire movie that goes for 12 hours on the cutting room floor.
I think that the writer had a great concept but no idea how to execute an actual storyline.
So disappointing. The special effects are fantastic but it was frustrating trying to make sense of how the characters 
got to the next scene because the previous scene stopped so abruptly. Would have be much better as a TV series.
Allow the story to be told in full.
Wait for the Streaming release.'''
]

In [264]:
# 텍스트 전처리
import re
reviews = map(lambda x: re.sub('[^A-Za-z]', ' ', x), reviews)

In [265]:
# feature 변환
reviews_cv = new_cvect.transform(reviews)
reviews_cv.shape

(2, 1454639)

In [266]:
# 예측
new_lrc.predict(reviews_cv)

array([1, 0], dtype=int64)